# Script permettant de tenter de corriger un mismatch en alignant le peptide human a la seq génomic et en cherchant le meilleur match
Input:   
    ID_file fichier de correspondance UniprotID - Ensembl Trasncript ID  
    Error_file fichier contenant toute les erreurs de type mismatch (script julie)  
    Exon_file fichier contenant les informations sur les positions des exons des transcripts  
    my_Genomic fichier fasta contenant les séquences génomiques des transcripts  
    my_cDNA fichier fasta contenant les séquences cDNA des transcripts  
    my_CDS fichier fasta contenant les séquences CDS des transcripts 

In [1]:
# Importer les lib et fonction
import json
import pandas as pd

def fasta2List(pathFasta):
    f = open(pathFasta, "r")
    title = []
    seq = []
    seq_temp = []
    for line in f:
        if line[0] == ">":
            seq.append(''.join(seq_temp).replace("\n", ""))
            title.append(line.replace("\n", ""))
            seq_temp = []
        else:
            seq_temp.append(line)
    seq.append(''.join(seq_temp).replace("\n", ""))
    seq.pop(0)
    dictionary = dict(zip(title, seq))
    return dictionary

In [2]:
# Importation de toutes les données utilisées
ID_file = pd.read_csv("../raw/uniprot-exon-map/transcript_ensembl.tab", sep = "\t")
Error_file = pd.read_csv("../raw/uniprot-exon-map/uniprot_new_errors_filt.txt", sep=" ", header=None)
Exon_file = pd.read_csv("../raw/uniprot-exon-map/Exon_map.tab", sep="\t", header=None)
my_Genomic = fasta2List("../raw/uniprot-exon-map/genomics_new.fa")
my_cDNA = fasta2List("../raw/uniprot-exon-map/cdna_seq.fa")
my_CDS = fasta2List("../raw/uniprot-exon-map/cds_new.fa")